In [ ]:
import os
import pyspark
conf = pyspark.SparkConf()

conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

In [ ]:
bake = spark.read.option('header',True).option('inferSchema',True).csv('hw2_data/Bakery.csv')

In [ ]:
from pyspark.sql.functions import *

In [ ]:
#"Q1"

In [ ]:
bakeSP = bake.withColumn('Date', split(bake['DateTime'], ' ').getItem(0)).withColumn('Time', split(bake['DateTime'], ' ').getItem(1))

In [ ]:
bakeSP1 = bakeSP.withColumn('Hour', split(bakeSP['Time'], ':').getItem(0))

In [ ]:
bk = bakeSP1.groupBy("Items","Date","Hour").agg(count("TransactionNo").alias("Count")).sort("Date","Hour").show(20,truncate=False)

In [ ]:
#"Q2"

In [ ]:
pt2 = bake.groupBy("Items","Daypart", "DayType").agg(count("TransactionNo").alias("Count"))

In [ ]:
pt2 = pt2.sort(col("Count").desc())

In [ ]:
from pyspark.sql.window import Window

In [ ]:
windowSpec  = Window.partitionBy("Daypart", "Daytype").orderBy(col("Count").desc())

In [ ]:
pt2.select('*', rank().over(windowSpec).alias('rank')).filter(col('rank') <= 3).show(1000) 

In [ ]:
#"Q3"

In [ ]:
restuarant = spark.read.option('header',True).option('inferSchema',True).options(delimiter=';').csv('hw2_data/Restaurants_in_Durham_County_NC.csv')

In [ ]:
restuarant.groupBy("Rpt_Area_Desc").agg(count("ID").alias("Count")).show()

In [ ]:
#"Q4"

In [ ]:
population = spark.read.option('inferSchema',True).csv('hw2_data/populationbycountry19802010millions.csv')

In [ ]:
population = population.withColumnRenamed("_c0","Region")

In [ ]:
countries= population.select('Region').rdd.map(lambda row: row.Region )
countries = countries.zipWithIndex()
country= countries.map(lambda x: (x[1],x[0]))

In [ ]:
#the loop

for i in range(2,32):
    rdd = population.rdd.map(lambda x: ((float(x[i])-float(x[i-1]))/float(x[i-1])*100)\
         if (x[i]!='NA'and x[i-1]!='NA'and x[i]!='--'and x[i-1]!='--') \
         else 0)
    
    mx = rdd.max()
    yr = 1979+i     
    print ("%i %s %f%%"% (yr,country.lookup(rdd.zipWithIndex().lookup(mx)[0])[0],mx))

In [ ]:
#"q5"

In [ ]:
wordCount =  sc.textFile('hw2_data/internet_archive_scifi_v3.txt')

In [ ]:
def lower_clean_str(x):
  punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
  lowercased_str = x.lower()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, '')
  return lowercased_str

In [ ]:
wordLower = wordCount.map(lower_clean_str)

In [ ]:
wordClean = wordLower.flatMap(lambda s: s.split(" ")).filter(lambda x:x!='')

In [ ]:
wordMap = wordClean.map(lambda word:(word,1))

In [ ]:
wordR = wordMap.reduceByKey(lambda x,y:(x+y)).sortByKey()

In [ ]:
wordR.take(25)

In [ ]:
#"Q6"

In [ ]:
wordClean2 = wordLower.flatMap(lambda s: s.split(".")).map(lambda s: s.split(" "))

In [ ]:
bigrams = wordClean2.flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:])))

In [ ]:
bigrams = bigrams.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

In [ ]:
bigrams.takeOrdered(10,lambda x: -x[1])